# RTB(RealTime Bidding)实时竞价系统

## 简介

RTB（Real Time Bidding）实时竞价，是一种利用第三方技术在数以百万计的网站或移动端针对每一个用户展示行为进行评估以及出价的竞价技术。与大量购买投放频次不同，实时竞价规避了无效的受众到达，针对有意义的用户进行购买。[百度百科](https://baike.baidu.com/item/RTB/3953125)

## 系统构成

RTB系统一般来说，由四个部分组成：

- DSP：（Demand Side Platform）需求方平台，可以简单的认为是广告主的集合；

- SSP：（Supply Side Platform）供给方平台，此类平台一般拥有自己的广告SDK和大量第三方合作伙伴，大量第三方应用集成广告SDK，下发广告；

- AdExchange：AdExchange是沟通需求方和供给方的平台；

- DMP：（Data Management Platform）DMP是提供数据的平台，对DSP（广告主）而言，从SSP（分发商）一侧获得的信息可能不足以支撑DSP（广告主）做出决策，DSP（广告主）可能会从DMP（数据平台）那里购买一些数据，DSP（广告主）报一个android id或者IDFA给DMP（数据平台），然后DMP（数据平台）返回相应的用户标签。

## RTB运行流程

![rtb](img/rtb_flow.png)

1. 打开集成了SSP广告SDK的应用（APP、网站、小程序等）同时上报（imei、应用名称、广告位、cookies等）信息；

2. SSP收到上报信息，向AdExchange发起出价请求（Bid Request）；

3. AdExchange将请求发送给此渠道对接的DSP，要求DSP出价；

4. DSP收到出价请求（Bid Request）取出携带的上报信息，请求DMP进行匹配。如果符合投放要求，则进行出价，返回出价响应（Bid Response），出价响应中会携带广告主投放的物料信息。如果不符合投放要求则不参与出价；

5. AdExchange收到0个或n（正整数）个出价响应（Bid Response），若规定时间内DSP没有返回响应，视为不参与出价。若无人出价则不下发广告，如有人出价则选出出价最高（前X个，0< X <= n）者视为竞价成功，预产生费用（此处涉及计费规则）；

6. AdExchange将竞价成功的DSP的物料信息等返回给SSP；

7. SSP通过SDK下发物料，应用展示广告；

8. SSP（可能通过AdExchange）向响应的DSP发送Win Notice，通知DSP该次Impression（广告展示）已经竞价成功至此RTB交易结束。

## RTB涉及规则算法

上述流程过程当中4和5分别涉及了一处算法和一处规则：

- 用户点击预测

- 计费规则

## CTR架构

主要包括两大部分：离线部分、在线部分，其中离线部分目标主要是训练出可用模型，而在线部分则考虑模型上线后，性能可能随时间而出现下降，若出现这种情况，可选择使用Online-Learning来在线更新模型：

### 离线部分

- 数据收集：主要收集和业务相关的数据，通常会有专门的同事在app位置进行埋点，拿到业务数据；
- 预处理：对埋点拿到的业务数据进行去脏去重；
- 构造数据集：经过预处理的业务数据，构造数据集，在切分训练、测试、验证集时应该合理根据业务逻辑来进行切分；
- 特征工程：对原始数据进行基本的特征处理，包括去除相关性大的特征，离散变量one-hot，连续特征离散化等等;
- 模型选择：选择合理的机器学习模型来完成相应工作，原则是先从简入深，先找到baseline，然后逐步优化；
- 超参选择：利用gridsearch、randomsearch或者hyperopt来进行超参选择，选择在离线数据集中性能最好的超参组合；
- 在线A/B Test：选择优化过后的模型和原先模型（如baseline）进行A/B Test，若性能有提升则替换原先模型；

### 在线部分

- Cache & Logic：设定简单过滤规则，过滤异常数据；
- 模型更新：当Cache & Logic 收集到合适大小数据时，对模型进行pretrain+finetuning，若在测试集上比原始模型性能高，则更新model server的模型参数；
- Model Server：接受数据请求，返回预测结果；